In [28]:
import pandas as pd
import warnings
from datetime import date

warnings.filterwarnings("ignore")

today = date.today().strftime("%m-%d-%Y")

active = pd.read_excel('W:\AP MM Service Request Log_COPY.xlsm', sheet_name = 'Active Materials', dtype=str)
selected_active_view = active[['Date Added', 'target sorg', 'target plant', 'email prefix\n(from request form)', 'SAP MATNR\n(from request form)', 'Service Requested\n(from request form)', 'Location\n(from request form)', 'Catalog', 'Ser', 'MTART/GenItemCat', ' sorg1k dchain', ' sorg1k cs', 'sorg1k price', ' sorg4k dchain', ' sorg4k cs', 'PGC', 'target sorg price', 'target sorg dchain', 'target sorg DWERK', 'target sorg cs', 'target sorg pub', 'target plant status', 'target plant mrp type', 'DWERK Plant Status', 'DWERK Plant Code', "mif/soerf check", 'Sales Text', 'INDIA GST\nINHTS', 'INDIA GST\nmarc.stuec', 'INDIA GST taxm1', 'STATUS_CHINA_ENERGY_LBL', "Regulatory Cert\n(Z62 Class)", 'Regulatory Cert\n(Z62 Characteristic)', 'Z62 characteristic\n(assigned in SAP)', 'PCE Assessment\n(received)', 'Date of PCE review', 'MIF Submitted', 'SOERF Submitted', 'pricing request', 'PCE cert rev req\'d', 'status', 'sort order']]

# req_local = pd.read_excel(r'..\RESOURCES\req_local.xlsx')
# req_gts = pd.read_excel(r'..\RESOURCES\req_gts.xlsx')
# list(req_gts.columns)
# list(req_local.columns)

In [37]:
###

### LIBS

import os
import platform
import warnings
import pandas as pd
from openpyxl import load_workbook
from openpyxl.formula.translate import Translator

warnings.filterwarnings("ignore")

### VARIABLES
ready_to_save = False

### OPEN LOG FILE NAD GENERATE SHEETS VARIABLES

# ACTUAL
log_file = 'W:\\AP MM Service Request Log.xlsm'
# TESTING
# log_file = 'W:\\AP MM Service Request Log_CLEANOUT.xlsm'
# log_file = 'W:\\AP MM Service Request Log_TESTING.xlsm'
# log_file = "W:\\VBA.xlsm"

log = load_workbook(filename=log_file, keep_vba=True)

ws_active=log['Active Materials']
ws_mara=log['mara']
ws_marc=log['marc']
ws_mvke=log['mvke']
ws_ausp=log['ausp']
ws_mlan=log['mlan']
ws_price=log['ZZ_MATPRC_HIST']
ws_gts=log['gts']
ws_text=log['sales text']
ws_list = [ws_mara, ws_marc, ws_mvke, ws_ausp, ws_mlan, ws_price, ws_gts, ws_text]

# # clean out data
for sheet in ws_list:
  sheet.delete_rows(100, ws_mara.max_row)
# ws_mara.delete_rows(100, ws_mara.max_row)
# ws_marc.delete_rows(100, ws_marc.max_row)
# ws_mvke.delete_rows(100, ws_mvke.max_row)
# ws_ausp.delete_rows(100, ws_ausp.max_row)
# ws_mlan.delete_rows(100, ws_mlan.max_row)
# ws_price.delete_rows(100, ws_price.max_row)
# ws_gts.delete_rows(100, ws_gts.max_row)
# ws_text.delete_rows(100, ws_text.max_row)

# load sap data to df
mara = pd.read_excel(r'W:\ap_data\mara.XLSX')
marc = pd.read_excel(r'W:\ap_data\marc.XLSX')
mvke = pd.read_excel(r'W:\ap_data\mvke.XLSX')
ausp = pd.read_excel(r'W:\ap_data\ausp.XLSX')
mlan = pd.read_excel(r'W:\ap_data\mlan.XLSX')
price = pd.read_excel(r'W:\ap_data\price.XLSX')
gts = pd.read_excel(r'W:\ap_data\gts.XLSX')
text = pd.read_excel(r'W:\ap_data\sales_text.xlsx')
inputs_list = [mara, marc, mvke, ausp, mlan, price, gts, text]


# output df to log
def populate_sheet(df, sheet):
  df = df.fillna('')
  output = []

  for index, row in df.iterrows():
    output.append(row.values.tolist())

  for rowy, row in enumerate(output, start = 2):
    for colx, value in enumerate(row, start = 2):
      sheet.cell(column = colx, row = rowy, value = value)

populate_sheet(mara, ws_mara)
populate_sheet(marc, ws_marc)
populate_sheet(mvke, ws_mvke)
populate_sheet(ausp, ws_ausp)
populate_sheet(mlan, ws_mlan)
populate_sheet(price, ws_price)
populate_sheet(gts, ws_gts)
populate_sheet(text, ws_text)


# EXTEND CONCATS
def extend_concats(sheet):
  last_row = sheet.max_row
  i = 100
  while i < last_row:
    i += 1
    formula = sheet['A2'].value
    sheet['A{}'.format(i)] = Translator(formula, origin="A2").translate_formula("A{}".format(i))

for sheet in ws_list:
  extend_concats(sheet)

# save test
log.save(r'..\OUTPUTS\clean_data.xlsm')

In [55]:
#MATNRs PRICE NEEDED
need_price = (selected_active_view['target sorg price'].isna()) & (~selected_active_view['SOERF Submitted'].isna()) & ((selected_active_view['status'].str.contains('cancel|complete|needs price;', case=False) == False) | (selected_active_view['status'].isnull()))

selected_active_view.loc[need_price, 'status'] = selected_active_view['status'].astype(str) + "needs price;"

price_requested = selected_active_view.loc[
  (selected_active_view['status'].str.contains('needs price;') == True)
]
print('\nMaterials needing PRICE in log:')
print(len(price_requested))

# MATNRs PCE NEEDED
# selected_active_view.loc[
#   (((selected_active_view['status'].str.contains('cancel|complete', case=False) == False)) & 
#   ((selected_active_view['status'].str.contains('pending PCE review;') == False)) & 
#   (selected_active_view['Service Requested\n(from request form)'] == 'Product Certification Review'))  |
#   ((selected_active_view['MTART/GenItemCat'].isin(['ZFG', 'ZTG'])) & (~selected_active_view['Regulatory Cert\n(Z62 Characteristic)'].isna()) & 
#   (selected_active_view['Z62 characteristic\n(assigned in SAP)'].isna()) & 
#   ((selected_active_view['status'].str.contains('cancel|complete', case=False) == False)) &
#   ((selected_active_view['status'].str.contains('pending PCE review;') == False))),
#   'status'
# ] = selected_active_view['status'] + 'pending PCE review;'

# pce_requested = selected_active_view.loc[
#   (selected_active_view['status'].str.contains('pending PCE review;') == True)
# ]
# print('\nMaterials needing PCE in log:')
# print(len(pce_requested))

# save test
# log.save(r'..\OUTPUTS\status_messages_am.xlsm')
need_price.to_excel(r'..\OUTPUTS\need_price.xlsx')
price_requested.to_excel(r'..\OUTPUTS\price_req.xlsx')
# pce_requested.to_excel(r'..\OUTPUTS\pce_req.xlsx')


Materials needing PRICE in log:
30
